In [1]:
import torch
from torch import nn
from torch.nn import init
import torch.utils.data as data_utils
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
SEED = 2019
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [5]:
dataset = np.loadtxt('../ml-1m/ratings.dat', delimiter='::', usecols=[0,1,3], dtype=int)
N_USER = np.max(dataset[:,0])
N_ITEM = np.max(dataset[:,1])

In [6]:
def generate_train_from_local(path, n_user, n_item):
    data = np.loadtxt(fname=path, delimiter="\t", skiprows=1, dtype=int)
    train_matrix = np.zeros((n_user, n_item), dtype = np.int8)
    for line in data:
        train_matrix[line[0],line[1]] = 1
    user_pos = dict()
    max_item_id = train_matrix.shape[1]
    max_item_num = 0
    for u, i in enumerate(train_matrix):
        pos_item = list(np.nonzero(i)[0])
        pos_item_num = len(pos_item)
        if  pos_item_num > max_item_num:
            max_item_num = pos_item_num
        user_pos[u] = pos_item
    train_user = list()
    train_item = list()
    for k in user_pos.keys():
        while len(user_pos[k]) < max_item_num:
            user_pos[k].append(max_item_id)
        train_user.append(k)
        train_item.append(user_pos[k])
    return np.array(train_user), np.array(train_item), train_matrix

def generate_test_from_local(path):
    data = np.loadtxt(fname=path, delimiter="\t", skiprows=1, dtype=int)
    return data

train_user, train_item, train_matrix = generate_train_from_local(path="../ml-1m/ml.train.txt",n_user=N_USER,n_item=N_ITEM)
test = generate_test_from_local(path="../ml-1m/ml.test.txt")

In [2]:
class ENMF(nn.Module):
    def __init__(self, emb_size, n_user, n_item, neg_weight, drop_out, count, c0=512, x=0.6):
        super().__init__()
        self.c0 = c0
        self.x  = x
        self.count = count
        self.n_user = n_user
        self.n_item = n_item
        self.neg_weight = neg_weight
        self.emb_size   = emb_size
        self.user_embs = nn.Embedding(n_user, emb_size)
        self.item_embs = nn.Embedding(n_item+1, emb_size)
        self.h = nn.Parameter(torch.randn(emb_size, 1))
        self.dropout = nn.Dropout(p=drop_out)
        self.freq = self.calcu_freq()
        self._reset_para()
        return
    
    def _reset_para(self):
        nn.init.xavier_normal_(self.user_embs.weight)
        nn.init.xavier_normal_(self.item_embs.weight)
        nn.init.constant_(self.h, 0.01)
        return
    
    def calcu_freq(self):
        freq_items = sorted(self.count.keys())
        freq_count = [self.count[k] for k in freq_items]
        freq = np.zeros(self.item_embs.weight.shape[0])
        freq[freq_items] = freq_count       
        #freq = freq/np.sum(freq)
        freq = np.power(freq, self.x)
        freq = self.c0 * freq/np.sum(freq)
        freq = torch.from_numpy(freq).type(torch.float).cuda()
        return freq
    
    def forward(self, uids, pos_iids):
        '''
        uids: B
        u_iids: B * L
        '''
        u_emb = self.dropout(self.user_embs(uids))
        pos_embs = self.item_embs(pos_iids)

        # torch.einsum("ab,abc->abc")
        # B * L * D
        mask = (~(pos_iids.eq(self.n_item))).float()
        pos_embs = pos_embs * mask.unsqueeze(2)

        # torch.einsum("ac,abc->abc")
        # B * L * D
        pq = u_emb.unsqueeze(1) * pos_embs
        # torch.einsum("ajk,kl->ajl")
        # B * L
        hpq = pq.matmul(self.h).squeeze(2)

        # loss
        pos_data_loss = torch.sum((1 - self.neg_weight) * hpq.square() - 2.0 * hpq)

        # torch.einsum("ab,ac->abc")
        part_1 = self.item_embs.weight.unsqueeze(2).bmm(self.item_embs.weight.unsqueeze(1))
        part_2 = u_emb.unsqueeze(2).bmm(u_emb.unsqueeze(1))

        # D * D
        part_1 = part_1.sum(0)
        part_2 = part_2.sum(0)
        part_3 = self.h.mm(self.h.t())
        all_data_loss = torch.sum(part_1 * part_2 * part_3)

        loss = self.neg_weight * all_data_loss + pos_data_loss
        return loss
    
    def rank(self, uid):
        '''
        uid: Batch_size
        '''
        uid_embs = self.user_embs(uid)
        user_all_items = uid_embs.unsqueeze(1) * self.item_embs.weight
        items_score = user_all_items.matmul(self.h).squeeze(2)
        return items_score
    
'''    def rank(self, user):
        res = self.user_embs(user).unsqueeze(0)
        res = res * self.item_embs.weight
        res = res.matmul(self.h).squeeze(1)
        return res'''

'    def rank(self, user):\n        res = self.user_embs(user).unsqueeze(0)\n        res = res * self.item_embs.weight\n        res = res.matmul(self.h).squeeze(1)\n        return res'

In [49]:
class NCF(nn.Module):
    def __init__(self, gmf_n_factors, layers,  n_user, n_item, activation = torch.relu, batch_normalization = False, n_output = 1):
        super(NCF, self).__init__()
        self.activation = activation
        self.do_bn = batch_normalization
        self.fcs = []
        self.bns = []
        self.n_layer  = len(layers)
        parameter_LeCun = np.sqrt(gmf_n_factors + layers[-1])

        #self.bn_userInput = nn.BatchNorm1d(1)   # for input data
        #self.bn_itemInput = nn.BatchNorm1d(1)   # for input data
        
        self.mlp_user_embedding_layer = nn.Embedding(n_user, int(layers[0]/2))
        self._set_normalInit(self.mlp_user_embedding_layer, hasBias = False) 
        self.mlp_item_embedding_layer = nn.Embedding(n_item, int(layers[0]/2))
        self._set_normalInit(self.mlp_item_embedding_layer, hasBias = False) 
        
        self.gmf_user_embedding_layer = nn.Embedding(n_user, gmf_n_factors)
        self._set_normalInit(self.gmf_user_embedding_layer, hasBias = False) 
        self.gmf_item_embedding_layer = nn.Embedding(n_item, gmf_n_factors)
        self._set_normalInit(self.gmf_item_embedding_layer, hasBias = False) 
        
        for i in range(1, self.n_layer):               # build hidden layers and BN layers
            fc = nn.Linear(layers[i-1], layers[i])
            self._set_normalInit(fc)                  # parameters initialization
            setattr(self, 'fc%i' % i, fc)       # IMPORTANT set layer to the Module
            self.fcs.append(fc)
            if self.do_bn:
                bn = nn.BatchNorm1d(layers[i])
                setattr(self, 'bn%i' % i, bn)   # IMPORTANT set layer to the Module
                self.bns.append(bn)

        self.predict = nn.Linear(gmf_n_factors + layers[-1], n_output)         # output layer
        self._set_uniformInit(self.predict, parameter = parameter_LeCun)            # parameters initialization
        return

    def _set_normalInit(self, layer, parameter = [0.0, 0.01], hasBias=True):
        init.normal_(layer.weight, mean = parameter[0], std = parameter[1])
        if hasBias:
            init.normal_(layer.bias, mean = parameter[0], std = parameter[1])
        return
    
    def _set_uniformInit(self, layer, parameter = 5, hasBias = True):
        init.uniform_(layer.weight, a = - parameter, b = parameter)
        if hasBias:
            init.uniform_(layer.bias, a = - parameter, b = parameter)
        return
    
    def _set_heNormalInit(self, layer, hasBias=True):
        init.kaiming_normal_(layer.weight, nonlinearity='relu')
        if hasBias:
            init.kaiming_normal_(layer.bias, nonlinearity='relu')
        return
    
    def _set_heUniformInit(self, layer, hasBias=True):
        init.kaiming_uniform_(layer.weight, nonlinearity='relu')
        if hasBias:
            init.kaiming_uniform_(layer.bias, nonlinearity='relu')
        return

    def forward(self, x1, x2):
        #if self.do_bn: 
            #x1 = self.bn_userInput(x1)     # input batch normalization
            #x2 = self.bn_itemInput(x2)
        mlp_x1 = self.mlp_user_embedding_layer(x1)
        mlp_x2 = self.mlp_item_embedding_layer(x2)
        
        gmf_x1 = self.gmf_user_embedding_layer(x1)
        gmf_x2 = self.gmf_item_embedding_layer(x2)
        
        mlp_x3 = torch.cat((mlp_x1, mlp_x2), dim=1)
        mlp_x  = torch.flatten(mlp_x3, start_dim=1)        
        for i in range(1, self.n_layer):
            mlp_x = self.fcs[i-1](mlp_x)
            if self.do_bn: 
                mlp_x = self.bns[i-1](mlp_x)   # batch normalization
            mlp_x = self.activation(mlp_x)
        
        gmf_x3 = torch.mul(gmf_x1, gmf_x2)
        gmf_x  = torch.flatten(gmf_x3, start_dim=1)

        x = torch.cat((mlp_x, gmf_x), dim=1)
        out = torch.sigmoid(self.predict(x))
        return out

In [20]:
class ConvNCF(nn.Module):
    def __init__(self, fm_sizes, n_user, n_item, n_fm, drop_out, myStride=2, n_output=1):
        ''' e.g.--> fm_sizes = [64,32,16,8,4,2,1] '''
        super(ConvNCF, self).__init__()
        self.convs = list()
        self.dropout = nn.Dropout(p=drop_out)
        self.user_embedding_layer = nn.Embedding(n_user, fm_sizes[0])
        self._set_normalInit(self.user_embedding_layer, hasBias = False) 
        #self._set_xavierInit(self.user_embedding_layer, hasBias = False)
        #self._set_heInit(self.user_embedding_layer, hasBias = False) 
        self.item_embedding_layer = nn.Embedding(n_item, fm_sizes[0])
        self._set_normalInit(self.item_embedding_layer, hasBias = False) 
        #self._set_xavierInit(self.item_embedding_layer, hasBias = False)
        #self._set_heInit(self.item_embedding_layer, hasBias = False) 
        for i in range(1, len(fm_sizes)):
            inChannel = 1 if i == 1 else n_fm
            #conv = nn.Conv2d(in_channels=inChannel, out_channels=32, kernel_size=fm_sizes[i]+myStride, stride=myStride)
            conv = nn.Conv2d(in_channels=inChannel, out_channels=n_fm, kernel_size=4, stride=myStride, padding=1)
            #self._set_normalInit(conv)
            #self._set_xavierInit(conv)
            self._set_heInit(conv)
            setattr(self, 'conv%i' % i, conv)
            self.convs.append(conv)

        self.predict = nn.Linear(n_fm, n_output)         # output layer
        self._set_xavierInit(self.predict)            # parameters initialization
        return
    
    def _set_xavierInit(self, layer, hasBias = True):
        init.xavier_uniform_(layer.weight)
        if hasBias:
            init.constant_(layer.bias, 0.01)
        return
    
    def _set_heInit(self, layer, hasBias = True):
        init.kaiming_normal_(layer.weight, nonlinearity='relu')
        if hasBias:
            init.constant_(layer.bias, 0.01)
        return
    
    def _set_normalInit(self, layer, parameter = [0.0, 0.1], hasBias = True):
        init.normal_(layer.weight, mean = parameter[0], std = parameter[1])
        if hasBias:
            init.constant_(layer.bias, 0.01)
        return
    
    def _set_uniformInit(self, layer, parameter = 1, hasBias = True):
        init.uniform_(layer.weight, a = 0, b = parameter)
        if hasBias:
            init.uniform_(layer.bias, a = 0, b = parameter)
        return
    
    def forward(self, user, item_pos, item_neg, train = True):
        user = self.user_embedding_layer(user)
        item_pos = self.item_embedding_layer(item_pos)
        if train:
            item_neg = self.item_embedding_layer(item_neg)
        x1, x2 = None, None
        temp1, temp2 = list(), list() 
        out1, out2 = None, None
        for i in range(user.size()[0]):
            temp1.append(torch.mm(user[i].T, item_pos[i]))
            if train:
                temp2.append(torch.mm(user[i].T, item_neg[i]))
        x1 = torch.stack(temp1)
        x1 = x1.view(x1.size()[0], -1, x1.size()[1], x1.size()[2])
        if train:
            x2 = torch.stack(temp2)
            x2 = x2.view(x2.size()[0], -1, x2.size()[1], x2.size()[2])
        ''' ## conv2d -input  (batch_size, channel, weight, height) '''
        for conv in self.convs:
            x1 = torch.relu(conv(x1))
            if train:
                x2 = torch.relu(conv(x2))
        ''' ## conv2d -output (batch_size, out_channel, out_weight, out_height) '''
        x1 = torch.flatten(x1, start_dim = 1)
        x1 = self.dropout(x1)
        if train:
            x2 = torch.flatten(x2, start_dim = 1)
            x2 = self.dropout(x2)
        #out1 = torch.sigmoid(self.dropout(self.predict(x1)))
        out1 = torch.sigmoid(self.predict(x1))
        if train:
            #out2 = torch.sigmoid(self.dropout(self.predict(x2)))
            out2 = torch.sigmoid(self.predict(x2))
        return out1, out2

In [35]:
def getHitRatio(ranklist, gtItem):
    #HR击中率，如果topk中有正例ID即认为正确
    if gtItem in ranklist:
        return 1
    return 0

def getNDCG(ranklist, gtItem):
    #NDCG归一化折损累计增益
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return np.log(2) / np.log(i+2)
    return 0

In [3]:
enmf = torch.load("./evalres/model/ENMF.pkl")

In [8]:
torch_test = torch.from_numpy(test).type(torch.LongTensor)
torch_testset = data_utils.TensorDataset(torch_test[:,0],torch_test[:,1])
test_loader = data_utils.DataLoader(dataset = torch_testset, batch_size = 128, num_workers = 0)

In [146]:
def enmf_rough(model, train_matrix, test_loader, topK = 100):
    n_users = train_matrix.shape[0]
    rank_all_users = list()
    model.eval()
    with torch.no_grad(): 
        for step, (batch_x, batch_y) in enumerate(test_loader):
            if torch.cuda.is_available():
                batch_x = batch_x.cuda()  
            prediction = model.rank(batch_x)
            pred_vector = -1 * (prediction.cpu().data.numpy())
            ranklist = np.argsort(pred_vector)
            for j, r in enumerate(ranklist):
                real_r = list()
                u = batch_x[j].cpu().data.numpy()
                i = 0
                while len(real_r) < topK:
                    if r[i]==train_matrix.shape[1]:
                        i += 1
                        continue
                    if train_matrix[u][r[i]] == 0:
                        real_r.append(r[i])
                    i += 1     
                rank_all_users.append(real_r)
    model.train()
    return np.array(rank_all_users)

In [147]:
rough_rank = enmf_rough(enmf, train_matrix, torch_test)

In [132]:
rough_rank.shape

(6040, 100)

In [133]:
def convNCF_fine(model, train_matrix, test, rough_rank, topK = 100):
    n_user = rough_rank.shape[0]
    n_item = rough_rank.shape[1]
    model.eval()
    rank_all_users = list()
    hit_list = list()
    undcg_list = list()
    with torch.no_grad():
        for u, rank in enumerate(rough_rank):
            item_list = torch.from_numpy(rank.reshape(-1, 1)).type(torch.LongTensor)
            user_list = torch.from_numpy(np.array([u for i in range(n_item)]).reshape(-1, 1)).type(torch.LongTensor)
            if torch.cuda.is_available():
                user_list, item_list = user_list.cuda(), item_list.cuda()
            prediction, _ = model(user_list, item_list, None, train = False)
            pred_vector = -1 * (prediction.cpu().data.numpy().reshape(-1))
            ranklist = rank[np.argsort(pred_vector)[:topK]]
            pos_item = test[u][1]
            rank_all_users.append(ranklist)
            hit_list.append(getHitRatio(ranklist, pos_item))
            undcg_list.append(getNDCG(ranklist, pos_item))
    model.train()
    hr = np.mean(hit_list)
    ndcg = np.mean(undcg_list)
    print('HR@', topK, ' = %.4f' %  hr)
    print('NDCG@', topK, ' = %.4f' % ndcg)
    return hr, ndcg, np.array(rank_all_users)

In [134]:
convNCF = torch.load("./evalres/model/ConvNCF.pkl")
hr, ndcg, rank_all_users = convNCF_fine(convNCF, train_matrix, test, rough_rank)

HR@ 100  = 0.4525
NDCG@ 100  = 0.0935


In [148]:
def ncf_fine(model, train_matrix, test, rough_rank, topK = 50):
    n_user = rough_rank.shape[0]
    n_item = rough_rank.shape[1]
    model.eval()
    rank_all_users = list()
    hit_list = list()
    undcg_list = list()
    with torch.no_grad():
        for u, rank in enumerate(rough_rank):
            item_list = torch.from_numpy(rank.reshape(-1, 1)).type(torch.LongTensor)
            user_list = torch.from_numpy(np.array([u for i in range(n_item)]).reshape(-1, 1)).type(torch.LongTensor)
            if torch.cuda.is_available():
                user_list, item_list = user_list.cuda(), item_list.cuda()
            prediction = model(user_list, item_list)
            pred_vector = -1 * (prediction.cpu().data.numpy().reshape(-1))
            ranklist = rank[np.argsort(pred_vector)[:topK]]
            pos_item = test[u][1]
            rank_all_users.append(ranklist)
            hit_list.append(getHitRatio(ranklist, pos_item))
            undcg_list.append(getNDCG(ranklist, pos_item))
    model.train()
    hr = np.mean(hit_list)
    ndcg = np.mean(undcg_list)
    print('HR@', topK, ' = %.4f' %  hr)
    print('NDCG@', topK, ' = %.4f' % ndcg)
    return hr, ndcg, np.array(rank_all_users)

In [149]:
ncf = torch.load("./evalres/model/NCF.pkl")
hr, ndcg, rank_all_users = ncf_fine(ncf, train_matrix, test, rough_rank)

HR@ 50  = 0.3012
NDCG@ 50  = 0.0885


In [142]:
HR@ 100  = 0.4525
NDCG@ 100  = 0.1103

SyntaxError: can't assign to operator (<ipython-input-142-f5ae4001f4a4>, line 1)

In [ ]:
HR@ 50  = 0.2753
NDCG@ 50  = 0.0817